# 7.10. PROJECT. Исследование поведения пользователей, зарегистрировавшихся в 2018 году

САМОСТОЯТЕЛЬНАЯ РАБОТА

Настало время для самостоятельной работы! Нам предстоит проверить два предположения:

    1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?
    
    2. Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем 
       сложности?

Проверку производим на основе данных пользователей, которые зарегистрировались в 2018 году (с 1 января по 31 декабря 2018 года включительно).

! ЗАМЕТКА :

В данном документе для удобства и лучшего визуального восприятия используется номерная разметка и текст формата Markdown ("не запущенный код" текста)

Для удобства проверки проекта можно перейти к разделу "Анализ временных промежутков". Анализ пользовательских событий и уникальных пользовательских путей был добавлен по желанию исполнителя (для собственного анализа, лучшего восприятия и ориентирования). Ответы на вопросы 1, 2, поставленные в проекте можно найти в пп. 6.5, 5.4, соответственно. 

## СБОР И ОБРАБОТКА ДАННЫХ

In [1]:
import pandas as pd

events_df = pd.read_csv('7_4_events.csv')

events_df.head(10)

,event_type,selected_level,start_time,tutorial_id,user_id,id
0,registration,NaN,2016-05-11T23:40:55,NaN,12583,28903
1,registration,NaN,2016-05-11T23:49:58,NaN,12584,28904
2,registration,NaN,2016-05-12T00:53:07,NaN,12585,28905
3,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585,28906
4,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585,28907
5,tutorial_start,NaN,2016-05-12T02:11:29,17563.0,12584,28908
6,tutorial_finish,NaN,2016-05-12T02:14:14,17563.0,12584,28909
7,level_choice,hard,2016-05-12T05:30:25,NaN,12584,28910
8,pack_choice,NaN,2016-05-12T05:38:24,NaN,12584,28911
9,tutorial_start,NaN,2016-05-12T07:19:01,17564.0,12583,28912


In [3]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   event_type      252334 non-null  object 
 1   selected_level  31086 non-null   object 
 2   start_time      252334 non-null  object 
 3   tutorial_id     125103 non-null  float64
 4   user_id         252334 non-null  int64  
 5   id              252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB


In [4]:
# Задаём условие: формируем список пользователей, зарегистрировавшихся в 2018 году

condition = (events_df.start_time >= '2018-01-01') & (events_df.start_time < '2019-01-01') & (events_df.event_type == 'registration')
registered = events_df[condition].user_id.to_list() # список пользователей, зарегистрировавшихся в 2018 г.
events = events_df[events_df.user_id.isin(registered)]

events.info() # ----- ТЕПЕРЬ БУДЕМ СЧИТАТЬ ЭТОТ датафрейм (events) ОСНОВНЫМ, нужным для дальнейшей работы ----- 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   event_type      66959 non-null  object 
 1   selected_level  8342 non-null   object 
 2   start_time      66959 non-null  object 
 3   tutorial_id     32954 non-null  float64
 4   user_id         66959 non-null  int64  
 5   id              66959 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 3.6+ MB


### ИССЛЕДУЕМ ПРОПУСКИ

In [5]:
# Наличие пропущенных строк обусловлено тем, что не все из параметров selected_level, tutorial_id присутствуют  
# в каждом из событий. Эти параметры заполнятся в зависимости от event_type. 
# К примеру, посмотрим на events_df, оставив в нём только такие строки, где event_type = level_choice:

events[events['event_type'] == 'level_choice'].info()

# Как видно, этот срез датафрейма не содержит пропущенных значений в столбце selected_level, но зато содержит пропуски
# в tutorial_id. Это связано с тем, что для событий типа level_choice не предусмотрена запись параметра tutorial_id.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 51424 to 118363
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   event_type      8342 non-null   object 
 1   selected_level  8342 non-null   object 
 2   start_time      8342 non-null   object 
 3   tutorial_id     0 non-null      float64
 4   user_id         8342 non-null   int64  
 5   id              8342 non-null   int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 456.2+ KB


In [7]:
# ----------------------------------------------------- СМОТРИМ ПОЛУЧЕННЫЙ ФАЙЛ: СРЕЗ ДАТАФРЕЙМА ПО ЗАДАННОМУ УСЛОВИЮ

events[events['event_type'] == 'level_choice']

,event_type,selected_level,start_time,tutorial_id,user_id,id
51424,level_choice,medium,2018-01-01T20:37:22,NaN,27835,80327
51428,level_choice,hard,2018-01-01T22:37:50,NaN,27839,80331
51441,level_choice,medium,2018-01-02T05:18:42,NaN,27840,80344
51443,level_choice,hard,2018-01-02T06:19:18,NaN,27845,80346
51450,level_choice,easy,2018-01-02T08:46:03,NaN,27842,80353
...,...,...,...,...,...,...
118337,level_choice,medium,2018-12-31T09:59:00,NaN,47732,147240
118342,level_choice,medium,2018-12-31T12:24:40,NaN,47747,147245
118353,level_choice,medium,2018-12-31T22:36:19,NaN,47753,147256
118362,level_choice,easy,2019-01-01T05:04:52,NaN,47755,147265


### ИССЛЕДУЕМ ЗНАЧЕНИЯ

##### Вызовем метод describe(),
    чтобы оценить характеристики каждого столбца. По умолчанию метод describe() выдаёт характеристики только по столбцам с численными типами (например, int64, float64). 
Мы вызовем этот метод с параметром
##### include='all'
    для того, чтобы отображать характеристики для всех столбцов.

In [8]:
events.describe(include = 'all')

,event_type,selected_level,start_time,tutorial_id,user_id,id
count,66959,8342,66959,32954.000000,66959.000000,66959.000000
unique,5,3,66809,NaN,NaN,NaN
top,registration,medium,2018-03-13T09:37:43,NaN,NaN,NaN
freq,19926,4645,9,NaN,NaN,NaN
mean,NaN,NaN,NaN,40532.934393,37781.543362,113787.000045
std,NaN,NaN,NaN,5213.486632,5751.497904,19329.542752
min,NaN,NaN,NaN,31505.000000,27832.000000,80308.000000
25%,NaN,NaN,NaN,36008.250000,32849.000000,97047.500000
50%,NaN,NaN,NaN,40529.500000,37719.000000,113787.000000
75%,NaN,NaN,NaN,45057.750000,42733.000000,130526.500000


In [9]:
##### Оценим, какие уникальные события есть в колонках event_type и selected_level:

events['event_type'].unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'pack_choice'], dtype=object)

In [10]:
events['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

In [11]:
# Также оценим, какое количество уникальных пользователей совершали события:

events['user_id'].nunique()

19926

##### Теперь проверим, как события, совершённые пользователями связаны с последующими оплатами пакетов вопросов. 

In [12]:
# Выведем первые 10 строк датафрейма purchase_df с помощью команды head():

purchase_df = pd.read_csv('7_4_Purchase.csv')

purchase_df.head(10)

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100
5,15679,12989,2016-05-14T02:19:20,200
6,15680,12684,2016-05-14T03:10:14,100
7,15681,12656,2016-05-14T03:42:48,100
8,15682,12624,2016-05-14T06:00:45,50
9,15683,13136,2016-05-14T07:31:03,50


In [13]:
# Воспользуемся методом info(), который позволяет получить общую информацию о датафрейме для того, 
# чтобы оценить, какие данные содержатся в датафрейме purchase_df:

purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


In [14]:
# Формируем датафрейм purchases_p - данные для пользователей, зарегистрировавшихся в 2018 году.

condition = (events_df.start_time >= '2018-01-01') & (events_df.start_time < '2019-01-01') & (events_df.event_type == 'registration')
registered = events_df[condition].user_id.to_list() # список пользователей, зарегистрировавшихся в 2018 г.
purchases_p = purchase_df[purchase_df.user_id.isin(registered)]

purchases_p.info()   # ----- ТЕПЕРЬ БУДЕМ СЧИТАТЬ ЭТОТ датафрейм (purchases) ОСНОВНЫМ, нужным для дальнейшей работы -----

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1600 non-null   int64 
 1   user_id         1600 non-null   int64 
 2   event_datetime  1600 non-null   object
 3   amount          1600 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 62.5+ KB


### ИССЛЕДУЕМ ЗНАЧЕНИЯ

In [15]:
# Снова обратимся к методу describe() для того, чтобы оценить характеристики каждого столбца датафрейма purchases_p:

purchases_p.describe()

,id,user_id,amount
count,1600.000000,1600.000000,1600.000000
mean,17645.505625,37752.766250,110.734375
std,462.038637,5822.621784,54.696628
min,16845.000000,27845.000000,25.000000
25%,17245.750000,32815.750000,50.000000
50%,17645.500000,37633.500000,100.000000
75%,18045.250000,43023.000000,150.000000
max,18452.000000,47742.000000,300.000000


## Анализ пользовательских событий

На данном этапе, исследуем поведение пользователей

    - Определим самые распространённые пути прохождения этапов в приложении.
    - Определим среднее время между различными этапами.
    
Это исследование позволит понять, как пользователи пользуются приложением,  
и сравнить фактический путь пользователей с тем, который был задуман при разработке приложения.

Когда мы поймём, какие этапы занимают больше всего времени, мы выясним, что требует оптимизации.

    - Период анализа — с 1 января по 31 декабря 2018 года.

Для того чтобы понимать, как пользователи переходят из этапа в этап и на каких из них возникают сложности,  
мы должны определить конверсию на каждом из этапов воронки. 
То есть нам нужно понять, какой процент пользователей переходит с предыдущего этапа на следующий.

In [16]:
# Определим какое количество пользователей, зарегистрировавшихся в 2018 г. совершает событие registration

display(events[events['event_type'] == 'registration']['user_id'].nunique())

19926

In [17]:
# Посмотрим на срез данных по событию level_choice :

events[events['event_type'] == 'level_choice'].head(10)

,event_type,selected_level,start_time,tutorial_id,user_id,id
51424,level_choice,medium,2018-01-01T20:37:22,NaN,27835,80327
51428,level_choice,hard,2018-01-01T22:37:50,NaN,27839,80331
51441,level_choice,medium,2018-01-02T05:18:42,NaN,27840,80344
51443,level_choice,hard,2018-01-02T06:19:18,NaN,27845,80346
51450,level_choice,easy,2018-01-02T08:46:03,NaN,27842,80353
51457,level_choice,easy,2018-01-02T11:53:11,NaN,27849,80360
51462,level_choice,medium,2018-01-02T14:09:58,NaN,27843,80365
51464,level_choice,medium,2018-01-02T15:10:27,NaN,27846,80367
51473,level_choice,medium,2018-01-02T18:09:02,NaN,27847,80376
51475,level_choice,medium,2018-01-02T18:39:44,NaN,27852,80378


In [18]:
# Посмотрим какое количество уникальных пользователей, зарегистрировавшихся в 2018 г. 
# переходят к выбору уровня сложности (level_choice) :

display(events[events['event_type'] == 'level_choice']['user_id'].nunique())

8342

##### Как мы видим, число пользователей, которые перешли к выбору уровня сложности (level_choice) - 8342, меньше, чем число пользователей, прошедших регистрацию - 19926. 

Рассчитаем долю пользователей, которые выбрали 
### уровень сложности (level_choice), 
от 
### общего числа зарегистрировавшихся в 2018 г. 
и запишем его в переменную 
##### percent_level_choice2018_users:

In [19]:
registered2018_users = events[events['event_type'] == 'registration']['user_id'].nunique()
level_choice2018_users = events[events['event_type'] == 'level_choice']['user_id'].nunique()
percent_level_registered_users2018 = level_choice2018_users / registered2018_users

print('Процент пользователей, выбравших уровень сложности (от общего числа зарегистрировавшихся в 2018 г.): {:.2%}'.format(percent_level_registered_users2018))

Процент пользователей, выбравших уровень сложности (от общего числа зарегистрировавшихся в 2018 г.): 41.86%


##### Таким образом, меньше половины пользователей (41,86 %) доходят до этапа выбора уровня сложности вопросов. 
Этот этап напрямую влияет на то, что пользователь будет пользоваться приложением через бесплатные возможности, которые в дальнейшем могут привести к оплате. Так что оптимизировать прохождение до этого этапа крайне важно для успешной монетизации приложения.

### Теперь оценим число пользователей, зарегистрировавшихся в 2018 г., которые купили пакеты вопросов.

In [20]:
# Для этого нам потребуется датафрейм purchases_p:

purchases2018_users = purchases_p['user_id'].nunique()

purchases2018_users

1600

Рассчитаем процент пользователей 
##### percent_of_paying_users
, которые 
### оплатили вопросы, 
от числа пользователей, которые 
### выбрали уровень сложности :

In [21]:
    ## level_choice2018_users = events[events['event_type'] == 'level_choice']['user_id'].nunique()
    
paying2018_users = purchases_p['user_id'].nunique()
percent_paying_level_users2018 = paying2018_users / level_choice2018_users
print('Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): {:.2%}'.format(percent_paying_level_users2018))

Процент пользователей, которые оплатили вопросы (от числа пользователей, которые выбрали уровень сложности): 19.18%


##### Таким образом, только 19.18% пользователей, которые выбирают уровень сложности, в дальнейшем покупают их.

Посмотрим, какой процент составляют 
### покупатели 
от общего числа 
### зарегистрировавшихся в 2018 г.

In [22]:
    ## registered2018_users = events[events['event_type'] == 'registration']['user_id'].nunique()
    
# Для этого создадим переменную percent_paying_registered_users2018:

percent_paying_registered_users2018 = paying2018_users / registered2018_users

print('Процент пользователей, которые оплатили вопросы (от числа зарегистрировавшихся пользователей в 2018 г.): {:.2%}'.format(percent_paying_registered_users2018))

Процент пользователей, которые оплатили вопросы (от числа зарегистрировавшихся пользователей в 2018 г.): 8.03%


##### Процент пользователей, зарегистрировавшихся в 2018 г., которые приобрели платный пакет, составляет 8.03% . 
Само по себе это число нам ни о чём не говорит, поэтому в рамках одной из гипотез мы рассмотрим этот процент для пользователей разных групп. И эта информация укажет нам на возможные пути оптимизации.

## Анализ уникальных пользовательских путей

Для начала давайте объединим датафрейм с событиями вместе с датафреймом по оплатам. Это позволит анализировать все эти события в рамках одной структуры данных.
##### Добавим в датафрейм 
### purchases_p
    столбец 
##### event_type, 
    который будет содержать одно значение 
##### purchase. 
Это нужно, чтобы в объединённом датафрейме однозначно выделить события оплаты.

In [23]:
# до добавления столбца event_type, содержащего значение purchase:

purchases_p

,id,user_id,event_datetime,amount
1171,16845,27845,2018-01-03T18:53:43,100
1172,16846,27865,2018-01-04T14:46:10,250
1174,16848,27911,2018-01-07T08:19:12,50
1175,16849,27910,2018-01-07T12:11:34,100
1176,16850,27940,2018-01-07T13:16:41,200
...,...,...,...,...
2767,18441,47498,2019-01-02T03:48:19,100
2768,18442,47647,2019-01-02T23:26:26,150
2769,18443,47554,2019-01-03T00:36:36,50
2774,18448,47742,2019-01-04T12:51:41,50


In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
# после добавления столбца event_type, содержащего значение purchase:

purchases_p['event_type'] = 'purchase'

purchases_p

,id,user_id,event_datetime,amount,event_type
1171,16845,27845,2018-01-03T18:53:43,100,purchase
1172,16846,27865,2018-01-04T14:46:10,250,purchase
1174,16848,27911,2018-01-07T08:19:12,50,purchase
1175,16849,27910,2018-01-07T12:11:34,100,purchase
1176,16850,27940,2018-01-07T13:16:41,200,purchase
...,...,...,...,...,...
2767,18441,47498,2019-01-02T03:48:19,100,purchase
2768,18442,47647,2019-01-02T23:26:26,150,purchase
2769,18443,47554,2019-01-03T00:36:36,50,purchase
2774,18448,47742,2019-01-04T12:51:41,50,purchase


##### Также у нас есть одинаковые столбцы id 
в двух датафреймах, но смысл их несколько отличается, так как 
##### столбец id в 
### events 
    указывает на идентификатор события, а 
##### столбец id в 
### purchases_p 
    указывает на идентификатор оплаты. 
Поэтому применим функцию 
### rename() 
##### для того, чтобы переименовать столбцы в датафреймах.

In [26]:
# до переименования столбца id в event_id, используя функцию rename() :

events.head()

,event_type,selected_level,start_time,tutorial_id,user_id,id
51405,registration,NaN,2018-01-01T03:48:40,NaN,27832,80308
51406,registration,NaN,2018-01-01T04:07:25,NaN,27833,80309
51407,registration,NaN,2018-01-01T08:35:10,NaN,27834,80310
51408,registration,NaN,2018-01-01T11:54:47,NaN,27835,80311
51409,registration,NaN,2018-01-01T13:28:07,NaN,27836,80312


In [27]:
# до переименования столбца id в purchase_id, используя функцию rename() :

purchases_p.head()

,id,user_id,event_datetime,amount,event_type
1171,16845,27845,2018-01-03T18:53:43,100,purchase
1172,16846,27865,2018-01-04T14:46:10,250,purchase
1174,16848,27911,2018-01-07T08:19:12,50,purchase
1175,16849,27910,2018-01-07T12:11:34,100,purchase
1176,16850,27940,2018-01-07T13:16:41,200,purchase


In [28]:
# после переименования столбцов id в двух датафреймах: id в event_id, id в purchase_id :

events = events.rename(columns = {'id': 'event_id'})
purchases_p = purchases_p.rename(columns = {'id': 'purchase_id'})

In [29]:
# после переименования столбца id в event_id, в датафрейме events :

events.head()

,event_type,selected_level,start_time,tutorial_id,user_id,event_id
51405,registration,NaN,2018-01-01T03:48:40,NaN,27832,80308
51406,registration,NaN,2018-01-01T04:07:25,NaN,27833,80309
51407,registration,NaN,2018-01-01T08:35:10,NaN,27834,80310
51408,registration,NaN,2018-01-01T11:54:47,NaN,27835,80311
51409,registration,NaN,2018-01-01T13:28:07,NaN,27836,80312


In [30]:
# после переименования столбца id в purchase_id, в датафрейме purchases_p :

purchases_p.head()

,purchase_id,user_id,event_datetime,amount,event_type
1171,16845,27845,2018-01-03T18:53:43,100,purchase
1172,16846,27865,2018-01-04T14:46:10,250,purchase
1174,16848,27911,2018-01-07T08:19:12,50,purchase
1175,16849,27910,2018-01-07T12:11:34,100,purchase
1176,16850,27940,2018-01-07T13:16:41,200,purchase


##### Следующим шагом объединим датафреймы 
### events и purchases_p 
с помощью функции 
#### pd.concat() 
и запишем объединенный датафрейм в переменную 
### total_events

In [31]:
total_events = pd.concat([events, purchases_p], sort = False)

# Посмотрим, что получилось:

total_events.head(10)

,event_type,selected_level,start_time,tutorial_id,user_id,event_id,purchase_id,event_datetime,amount
51405,registration,NaN,2018-01-01T03:48:40,NaN,27832,80308.0,NaN,NaN,NaN
51406,registration,NaN,2018-01-01T04:07:25,NaN,27833,80309.0,NaN,NaN,NaN
51407,registration,NaN,2018-01-01T08:35:10,NaN,27834,80310.0,NaN,NaN,NaN
51408,registration,NaN,2018-01-01T11:54:47,NaN,27835,80311.0,NaN,NaN,NaN
51409,registration,NaN,2018-01-01T13:28:07,NaN,27836,80312.0,NaN,NaN,NaN
51410,registration,NaN,2018-01-01T14:08:40,NaN,27837,80313.0,NaN,NaN,NaN
51411,registration,NaN,2018-01-01T14:42:58,NaN,27838,80314.0,NaN,NaN,NaN
51412,tutorial_start,NaN,2018-01-01T14:54:40,31505.0,27836,80315.0,NaN,NaN,NaN
51413,tutorial_start,NaN,2018-01-01T15:00:51,31506.0,27835,80316.0,NaN,NaN,NaN
51414,tutorial_finish,NaN,2018-01-01T15:06:15,31506.0,27835,80317.0,NaN,NaN,NaN


In [32]:
total_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   event_type      68559 non-null  object 
 1   selected_level  8342 non-null   object 
 2   start_time      66959 non-null  object 
 3   tutorial_id     32954 non-null  float64
 4   user_id         68559 non-null  int64  
 5   event_id        66959 non-null  float64
 6   purchase_id     1600 non-null   float64
 7   event_datetime  1600 non-null   object 
 8   amount          1600 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 5.2+ MB


##### Теперь сбросим индексы объединенного датафрейма 
    ##### (так как после объединения они дублировались и несут мало смысла) 
### с помощью метода reset_index() 
##### и отсортируем все события по возрастанию времени с помощью 
### sort_values():

In [33]:
# до сброса индексов объединенного датафрейма с помощью метода reset_index():

total_events

,event_type,selected_level,start_time,tutorial_id,user_id,event_id,purchase_id,event_datetime,amount
51405,registration,NaN,2018-01-01T03:48:40,NaN,27832,80308.0,NaN,NaN,NaN
51406,registration,NaN,2018-01-01T04:07:25,NaN,27833,80309.0,NaN,NaN,NaN
51407,registration,NaN,2018-01-01T08:35:10,NaN,27834,80310.0,NaN,NaN,NaN
51408,registration,NaN,2018-01-01T11:54:47,NaN,27835,80311.0,NaN,NaN,NaN
51409,registration,NaN,2018-01-01T13:28:07,NaN,27836,80312.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2767,purchase,NaN,NaN,NaN,47498,NaN,18441.0,2019-01-02T03:48:19,100.0
2768,purchase,NaN,NaN,NaN,47647,NaN,18442.0,2019-01-02T23:26:26,150.0
2769,purchase,NaN,NaN,NaN,47554,NaN,18443.0,2019-01-03T00:36:36,50.0
2774,purchase,NaN,NaN,NaN,47742,NaN,18448.0,2019-01-04T12:51:41,50.0


In [34]:
# после сброса индексов объединенного датафрейма с помощью метода reset_index():

total_events = total_events.reset_index(drop = True).sort_values('start_time')

total_events

,event_type,selected_level,start_time,tutorial_id,user_id,event_id,purchase_id,event_datetime,amount
0,registration,NaN,2018-01-01T03:48:40,NaN,27832,80308.0,NaN,NaN,NaN
1,registration,NaN,2018-01-01T04:07:25,NaN,27833,80309.0,NaN,NaN,NaN
2,registration,NaN,2018-01-01T08:35:10,NaN,27834,80310.0,NaN,NaN,NaN
3,registration,NaN,2018-01-01T11:54:47,NaN,27835,80311.0,NaN,NaN,NaN
4,registration,NaN,2018-01-01T13:28:07,NaN,27836,80312.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
68554,purchase,NaN,NaN,NaN,47498,NaN,18441.0,2019-01-02T03:48:19,100.0
68555,purchase,NaN,NaN,NaN,47647,NaN,18442.0,2019-01-02T23:26:26,150.0
68556,purchase,NaN,NaN,NaN,47554,NaN,18443.0,2019-01-03T00:36:36,50.0
68557,purchase,NaN,NaN,NaN,47742,NaN,18448.0,2019-01-04T12:51:41,50.0


In [35]:
total_events.event_type = total_events.event_type.astype(str)

### ИТАК, МЫ СФОРМИРОВАЛИ ВЫБОРКУ, В КОТОРОЙ ОБЪЕДИНИЛИ СОБЫТИЯ И ОПЛАТЫ (ПОКУПКИ ПАКЕТОВ ПЛАТНЫХ ВОПРОСОВ)!

##### Теперь можно для каждого пользователя создать список, который будет содержать 
##### во временной последовательности все события, совершаемые данным пользователем.

Для этого воспользуемся методом 
### groupby по столбцу event_type 
и применим аргегирующую функцию 
#### apply(list). 
    ##### Таким образом, мы сгруппируем строки по пользователю, 
    ##### а затем объединим в списки содержимое столбца 
#### event_type по каждому пользователю.
Запишем результат в датафрейм 
### user_path2018:

In [36]:
user_path2018 = total_events.groupby(['user_id'])['event_type'].apply(list).reset_index()

user_path2018.head(10)

,user_id,event_type
0,27832,[registration]
1,27833,"[registration, tutorial_start, tutorial_finish]"
2,27834,"[registration, tutorial_start, tutorial_finish]"
3,27835,"[registration, tutorial_start, tutorial_finish..."
4,27836,"[registration, tutorial_start, tutorial_start,..."
5,27837,[registration]
6,27838,[registration]
7,27839,"[registration, tutorial_start, tutorial_finish..."
8,27840,"[registration, tutorial_start, level_choice]"
9,27841,"[registration, tutorial_start, tutorial_finish]"


##### Как видно, теперь у нас есть датафрейм, где для каждого пользователя в столбце 
### event_type 
    указаны последовательно все его события — пути пользователя. 

Если мы объединим данные по этой колонке и подсчитаем количество пользователей по каждому пути, то получим 
##### наиболее популярные пути пользователей. 

In [37]:
# Прежде чем мы сможем оценить популярные пути пользователей, преобразуем список событий в строку event_path 
# Эта операция нужна для оптимизации скорости объединения, так как иначе pandas может делать подсчёт слишком долго.

user_path2018['event_path'] = user_path2018['event_type'].apply(lambda x: '>'.join(x))
user_path2018['event_path'].head()

0                                         registration
1          registration>tutorial_start>tutorial_finish
2          registration>tutorial_start>tutorial_finish
3    registration>tutorial_start>tutorial_finish>le...
4    registration>tutorial_start>tutorial_start>tut...
Name: event_path, dtype: object

In [38]:
# Теперь можно сгруппировать датафрейм user_paths2018 по столбцу event_path, подсчитав число пользователей:

user_paths2018 = user_path2018.groupby(['event_path'])['user_id'].nunique().sort_values(ascending = False)

# Выведем 10 самых популярных последовательностей:

user_paths2018.head(10)

event_path
registration                                                                                           7970
registration>tutorial_start>tutorial_finish>level_choice>pack_choice                                   2796
registration>tutorial_start>tutorial_finish                                                            1956
registration>tutorial_start>tutorial_finish>level_choice                                               1713
registration>tutorial_start>tutorial_finish>level_choice>pack_choice>purchase                          1083
registration>tutorial_start                                                                             842
registration>tutorial_start>level_choice>pack_choice                                                    346
registration>tutorial_start>tutorial_finish>tutorial_start>tutorial_finish                              323
registration>tutorial_start>level_choice                                                                243
registration>tuto

##### Как мы видим, среди 10 самых популярных последовательностей только одна содержит этап оплаты. 
Это последовательность 
### registration > tutorial_start > tutorial_finish > level_choice > pack_choice > purchase. 

In [39]:
# Посмотрим, какие ещё последовательности содержат в себе оплату:

display(user_paths2018[user_paths2018.index.str.contains('purchase')].head(10))

event_path
registration>tutorial_start>tutorial_finish>level_choice>pack_choice>purchase                                                                                                 1083
registration>tutorial_start>level_choice>pack_choice>purchase                                                                                                                  124
registration>tutorial_start>tutorial_finish>level_choice>pack_choice>tutorial_start>tutorial_finish>purchase                                                                   101
registration>tutorial_start>tutorial_finish>tutorial_start>tutorial_finish>level_choice>pack_choice>purchase                                                                    52
registration>tutorial_start>tutorial_start>tutorial_finish>level_choice>pack_choice>purchase                                                                                    26
registration>level_choice>pack_choice>purchase                                                

##### Исследование уникальных пользовательских путей показывает, что большинство последовательностей, которые содержат в себе оплату, также содержат выбор уровня сложности (level_choice). 

### Это наводит нас на гипотезу, что вероятность оплаты может зависеть от того, выбирал ли пользователь уровень сложности. 

Проверим это в рамках следующей гипотезы. >>> 

## Анализ временных промежутков

##### Определим, какое время проходит между (1) регистрацией (registration) и (2) выбором уровня сложности (level_choice) для разных групп пользователей с разным уровнем сложности

# 1.1
Первым делом выделим отдельный датафрейм 
### registration2018
, который будет содержать только события с 
##### event_type = registration

In [40]:
registration2018 = total_events[total_events['event_type'] == 'registration']

registration2018

,event_type,selected_level,start_time,tutorial_id,user_id,event_id,purchase_id,event_datetime,amount
0,registration,NaN,2018-01-01T03:48:40,NaN,27832,80308.0,NaN,NaN,NaN
1,registration,NaN,2018-01-01T04:07:25,NaN,27833,80309.0,NaN,NaN,NaN
2,registration,NaN,2018-01-01T08:35:10,NaN,27834,80310.0,NaN,NaN,NaN
3,registration,NaN,2018-01-01T11:54:47,NaN,27835,80311.0,NaN,NaN,NaN
4,registration,NaN,2018-01-01T13:28:07,NaN,27836,80312.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
66941,registration,NaN,2018-12-31T18:58:55,NaN,47753,147249.0,NaN,NaN,NaN
66942,registration,NaN,2018-12-31T19:14:08,NaN,47754,147250.0,NaN,NaN,NaN
66947,registration,NaN,2018-12-31T21:15:14,NaN,47755,147255.0,NaN,NaN,NaN
66950,registration,NaN,2018-12-31T23:17:30,NaN,47756,147258.0,NaN,NaN,NaN


# 1.2

Проверим, что в нашем датафрейме 
### registration2018 
для каждого пользователя содержится только по одному событию, то есть по одной регистрации на пользователя.  
Это важно, чтобы однозначно для одного пользователя присоединить только одно событие, по которому мы будем считать время.

Наличие более чем одной регистрации на пользователя может говорить о технической ошибке: например, какие-то события записываются в базу по несколько раз. 
##### Для определения среднего количества событий на пользователя мы можем воспользоваться 
###  методом value_counts(), 
результатом которого будет серия с количеством событий на каждого пользователя. 
##### Затем, усреднив эти значения между собой, мы получим среднее число событий типа registration на одного пользователя:

In [41]:
registration2018['user_id'].value_counts().mean()

1.0

# 1.3

##### Следующим шагом оставим в датафрейме 
### registration2018 
только те данные, которые нужны для наших вычислений — 
##### столбец user_id 
                    с идентификатором пользователя  
##### и столбец start_time 
                    со временем регистрации. 

Также переименуем столбец 
#### start_time 
      в 
#### столбец registration_time
      для понятности.

In [42]:
registration2018 = registration2018[['user_id', 'start_time']].rename(columns = {'start_time': 'registration_time'})

registration2018

,user_id,registration_time
0,27832,2018-01-01T03:48:40
1,27833,2018-01-01T04:07:25
2,27834,2018-01-01T08:35:10
3,27835,2018-01-01T11:54:47
4,27836,2018-01-01T13:28:07
...,...,...
66941,47753,2018-12-31T18:58:55
66942,47754,2018-12-31T19:14:08
66947,47755,2018-12-31T21:15:14
66950,47756,2018-12-31T23:17:30


# 2.1

##### Теперь возьмемся за следующее событие - выбор уровня сложности (level_choice).

##### Выделим отдельный датафрейм 
### level_choice2018
    ## который будет содержать только события с 
##### event_type = level_choice

In [43]:
level_choice2018 = total_events[total_events['event_type'] == 'level_choice']

level_choice2018

,event_type,selected_level,start_time,tutorial_id,user_id,event_id,purchase_id,event_datetime,amount
19,level_choice,medium,2018-01-01T20:37:22,NaN,27835,80327.0,NaN,NaN,NaN
23,level_choice,hard,2018-01-01T22:37:50,NaN,27839,80331.0,NaN,NaN,NaN
36,level_choice,medium,2018-01-02T05:18:42,NaN,27840,80344.0,NaN,NaN,NaN
38,level_choice,hard,2018-01-02T06:19:18,NaN,27845,80346.0,NaN,NaN,NaN
45,level_choice,easy,2018-01-02T08:46:03,NaN,27842,80353.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
66932,level_choice,medium,2018-12-31T09:59:00,NaN,47732,147240.0,NaN,NaN,NaN
66937,level_choice,medium,2018-12-31T12:24:40,NaN,47747,147245.0,NaN,NaN,NaN
66948,level_choice,medium,2018-12-31T22:36:19,NaN,47753,147256.0,NaN,NaN,NaN
66956,level_choice,easy,2019-01-01T05:04:52,NaN,47755,147265.0,NaN,NaN,NaN


# 2.2

Проверим, что в нашем датафрейме 
### level_choice2018 
для каждого пользователя содержится только по одному событию, то есть по одному выбору уровня сложности на пользователя.  
Это важно, чтобы однозначно для одного пользователя присоединить только одно событие, по которому мы будем считать время.

Наличие более чем одного выбора сложности на пользователя может говорить о технической ошибке: например, какие-то события записываются в базу по несколько раз. 
##### Для определения среднего количества событий на пользователя мы можем воспользоваться 
###  методом value_counts(), 
результатом которого будет серия с количеством событий на каждого пользователя. 
##### Затем, усреднив эти значения между собой, мы получим среднее число событий типа level_choice на одного пользователя:

In [44]:
level_choice2018['user_id'].value_counts().mean()

# Итак, среднее число событий выбора уровня сложности на одного пользователя равняется 1.

1.0

# 2.3

##### Следующим шагом оставим в датафрейме 
### level_choice2018 
только те данные, которые нужны для наших вычислений — 
##### столбец user_id 
                    с идентификатором пользователя  
##### столбец selected_level 
                    с уровнем сложности (easy/medium/hard) и
##### столбец start_time 
                    со временем выбора уровня сложности.

Также переименуем столбец 
#### start_time 
      в 
#### столбец level_choice_time
      для понятности.

In [45]:
level_choice2018 = level_choice2018[['user_id', 'selected_level', 'start_time']].rename(columns = {'start_time': 'level_choice_time'})

level_choice2018

,user_id,selected_level,level_choice_time
19,27835,medium,2018-01-01T20:37:22
23,27839,hard,2018-01-01T22:37:50
36,27840,medium,2018-01-02T05:18:42
38,27845,hard,2018-01-02T06:19:18
45,27842,easy,2018-01-02T08:46:03
...,...,...,...
66932,47732,medium,2018-12-31T09:59:00
66937,47747,medium,2018-12-31T12:24:40
66948,47753,medium,2018-12-31T22:36:19
66956,47755,easy,2019-01-01T05:04:52


# 3.1

###### Настало время объединить между собой данные двух получившихся датафреймов (1 + 2) : 
#### registration2018 и level_choice2018


##### Объединение позволит для каждого пользователя в рамках одного датафрейма 
### указать и время регистрации и время выбора уровня сложности.


Объединить датафреймы мы можем с помощью функции 
#### merge(), объединяя данные по параметру user_id.

Также нам нужно объединить данные с использованием параметра 
##### how='inner'
, это позволит в объединённом датафрейме оставить только такие идентификаторы пользователей, которые есть в обоих датафреймах.

In [46]:
merged2018_registration_level = registration2018.merge(level_choice2018, on = 'user_id', how = 'inner')

merged2018_registration_level

,user_id,registration_time,selected_level,level_choice_time
0,27835,2018-01-01T11:54:47,medium,2018-01-01T20:37:22
1,27839,2018-01-01T18:24:01,hard,2018-01-01T22:37:50
2,27840,2018-01-01T20:53:21,medium,2018-01-02T05:18:42
3,27842,2018-01-01T23:18:46,easy,2018-01-02T08:46:03
4,27843,2018-01-02T00:02:28,medium,2018-01-02T14:09:58
...,...,...,...,...
8337,47744,2018-12-30T22:12:24,easy,2018-12-31T07:53:04
8338,47747,2018-12-31T03:01:17,medium,2018-12-31T12:24:40
8339,47753,2018-12-31T18:58:55,medium,2018-12-31T22:36:19
8340,47755,2018-12-31T21:15:14,easy,2019-01-01T05:04:52


### Теперь, получим из объединённого датафрейма данные по каждому пользователю, исходя из выбранного им уровня сложности (level_choice) - easy / medium / hard :

## 3.1.1
##### Сначала рассмотрим данные по уровню сложности - easy

In [47]:
registration_to_easy_level = merged2018_registration_level[merged2018_registration_level['selected_level'] == 'easy']

registration_to_easy_level

,user_id,registration_time,selected_level,level_choice_time
3,27842,2018-01-01T23:18:46,easy,2018-01-02T08:46:03
8,27849,2018-01-02T07:03:55,easy,2018-01-02T11:53:11
10,27853,2018-01-02T15:47:14,easy,2018-01-03T00:39:54
14,27859,2018-01-02T21:30:27,easy,2018-01-03T01:11:45
17,27863,2018-01-03T03:53:48,easy,2018-01-03T09:31:00
...,...,...,...,...
8329,47725,2018-12-30T10:39:18,easy,2018-12-30T12:28:11
8332,47733,2018-12-30T14:44:47,easy,2018-12-30T19:44:52
8336,47743,2018-12-30T21:27:31,easy,2018-12-31T02:23:50
8337,47744,2018-12-30T22:12:24,easy,2018-12-31T07:53:04


## 3.1.2
##### Рассмотрим данные по уровню сложности - medium

In [48]:
registration_to_medium_level = merged2018_registration_level[merged2018_registration_level['selected_level'] == 'medium']

registration_to_medium_level

,user_id,registration_time,selected_level,level_choice_time
0,27835,2018-01-01T11:54:47,medium,2018-01-01T20:37:22
2,27840,2018-01-01T20:53:21,medium,2018-01-02T05:18:42
4,27843,2018-01-02T00:02:28,medium,2018-01-02T14:09:58
6,27846,2018-01-02T05:07:52,medium,2018-01-02T15:10:27
7,27847,2018-01-02T05:13:08,medium,2018-01-02T18:09:02
...,...,...,...,...
8334,47741,2018-12-30T20:41:06,medium,2018-12-31T02:01:26
8335,47742,2018-12-30T20:48:42,medium,2018-12-31T04:58:15
8338,47747,2018-12-31T03:01:17,medium,2018-12-31T12:24:40
8339,47753,2018-12-31T18:58:55,medium,2018-12-31T22:36:19


## 3.1.3
##### Рассмотрим данные по уровню сложности - hard

In [49]:
registration_to_hard_level = merged2018_registration_level[merged2018_registration_level['selected_level'] == 'hard']

registration_to_hard_level

,user_id,registration_time,selected_level,level_choice_time
1,27839,2018-01-01T18:24:01,hard,2018-01-01T22:37:50
5,27845,2018-01-02T01:35:56,hard,2018-01-02T06:19:18
18,27865,2018-01-03T11:14:57,hard,2018-01-04T05:56:32
26,27888,2018-01-04T13:05:23,hard,2018-01-04T19:41:29
31,27910,2018-01-05T10:45:33,hard,2018-01-05T11:59:50
...,...,...,...,...
8282,47611,2018-12-27T18:45:30,hard,2018-12-28T12:10:19
8285,47619,2018-12-27T20:17:35,hard,2018-12-27T22:57:39
8319,47700,2018-12-29T13:10:26,hard,2018-12-30T04:13:39
8321,47705,2018-12-29T14:19:44,hard,2018-12-29T18:44:23


##### Проверим правильность разделения пользователей на группы, просуммировав длины строк всех выделенных групп по выбранному уровню сложности (easy / medium / hard) :

In [50]:
len(registration_to_easy_level) + len(registration_to_medium_level) + len(registration_to_hard_level) == len(merged2018_registration_level)

True

# 4.1

##### Теперь возьмемся за следующее событие - покупка платных пакетов вопросов (purchase).

##### Выделим отдельный датафрейм 
### purchase2018
    ## который будет содержать только события с 
##### event_type = purchase

In [51]:
purchase2018 = total_events[total_events['event_type'] == 'purchase']

purchase2018

,event_type,selected_level,start_time,tutorial_id,user_id,event_id,purchase_id,event_datetime,amount
66959,purchase,NaN,NaN,NaN,27845,NaN,16845.0,2018-01-03T18:53:43,100.0
66960,purchase,NaN,NaN,NaN,27865,NaN,16846.0,2018-01-04T14:46:10,250.0
66961,purchase,NaN,NaN,NaN,27911,NaN,16848.0,2018-01-07T08:19:12,50.0
66962,purchase,NaN,NaN,NaN,27910,NaN,16849.0,2018-01-07T12:11:34,100.0
66963,purchase,NaN,NaN,NaN,27940,NaN,16850.0,2018-01-07T13:16:41,200.0
...,...,...,...,...,...,...,...,...,...
68554,purchase,NaN,NaN,NaN,47498,NaN,18441.0,2019-01-02T03:48:19,100.0
68555,purchase,NaN,NaN,NaN,47647,NaN,18442.0,2019-01-02T23:26:26,150.0
68556,purchase,NaN,NaN,NaN,47554,NaN,18443.0,2019-01-03T00:36:36,50.0
68557,purchase,NaN,NaN,NaN,47742,NaN,18448.0,2019-01-04T12:51:41,50.0


# 4.2

##### Так же, как и в случае с выбором уровня сложности (level_choice), посчитаем, сколько покупок платных пакетов вопросов (purchase) приходится на пользователя:

In [52]:
purchase2018['user_id'].value_counts().mean()

1.0

# 4.3

Так же, как и в случае с датафреймом 
#### level_choice2018, 
    ##### оставим только такие столбцы, которые пригодятся нам в дальнейшем. 
    Это столбцы 
### user_id, purchase_id, event_datetime

(Зачем нам понадобится purchase_id, узнаем чуть дальше.)

Также переименуем колонку 
### event_datetime 
в 
### purchase_time :

In [53]:
purchase2018 = purchase2018[['user_id', 'purchase_id', 'event_datetime']].rename(columns = {'event_datetime': 'purchase_time'})

purchase2018

,user_id,purchase_id,purchase_time
66959,27845,16845.0,2018-01-03T18:53:43
66960,27865,16846.0,2018-01-04T14:46:10
66961,27911,16848.0,2018-01-07T08:19:12
66962,27910,16849.0,2018-01-07T12:11:34
66963,27940,16850.0,2018-01-07T13:16:41
...,...,...,...
68554,47498,18441.0,2019-01-02T03:48:19
68555,47647,18442.0,2019-01-02T23:26:26
68556,47554,18443.0,2019-01-03T00:36:36
68557,47742,18448.0,2019-01-04T12:51:41


# 5.1

###### Настало время объединить между собой данные двух получившихся датафреймов (3 + 4) : 
#### merged2018_registration_level и purchase2018


##### Объединение позволит для каждого пользователя в рамках одного датафрейма 
### посмотреть время от регистрации до выбора уровня сложности и до времени покупки платных пакетов вопросов.


Объединить датафреймы мы можем с помощью функции 
#### merge(), объединяя данные по параметру user_id.

Также нам нужно объединить данные с использованием параметра 
##### how='inner'
, это позволит в объединённом датафрейме оставить только такие идентификаторы пользователей, которые есть в обоих датафреймах.

In [54]:
merged2018_registration_level_purchase = merged2018_registration_level.merge(purchase2018, on = 'user_id', how = 'inner')

merged2018_registration_level_purchase

,user_id,registration_time,selected_level,level_choice_time,purchase_id,purchase_time
0,27845,2018-01-02T01:35:56,hard,2018-01-02T06:19:18,16845.0,2018-01-03T18:53:43
1,27865,2018-01-03T11:14:57,hard,2018-01-04T05:56:32,16846.0,2018-01-04T14:46:10
2,27884,2018-01-04T11:50:43,easy,2018-01-04T16:18:39,16854.0,2018-01-08T19:37:34
3,27910,2018-01-05T10:45:33,hard,2018-01-05T11:59:50,16849.0,2018-01-07T12:11:34
4,27911,2018-01-05T10:48:24,hard,2018-01-05T17:39:02,16848.0,2018-01-07T08:19:12
...,...,...,...,...,...,...
1595,47671,2018-12-28T20:48:59,easy,2018-12-29T04:26:01,18429.0,2018-12-30T01:39:50
1596,47687,2018-12-29T09:49:24,easy,2018-12-29T11:37:35,18432.0,2018-12-30T09:05:28
1597,47712,2018-12-29T18:21:28,medium,2018-12-29T22:44:06,18434.0,2018-12-31T09:05:20
1598,47732,2018-12-30T14:20:21,medium,2018-12-31T09:59:00,18452.0,2019-01-06T07:14:25


### Теперь, получим из объединённого датафрейма данные по каждому пользователю, исходя из выбранного им уровня сложности (level_choice) easy / medium / hard :

## 5.1.1
##### Сначала рассмотрим данные по уровню сложности - easy

In [55]:
easy_level_to_purchase = merged2018_registration_level_purchase[merged2018_registration_level_purchase['selected_level'] == 'easy']

easy_level_to_purchase

,user_id,registration_time,selected_level,level_choice_time,purchase_id,purchase_time
2,27884,2018-01-04T11:50:43,easy,2018-01-04T16:18:39,16854.0,2018-01-08T19:37:34
20,28090,2018-01-09T19:31:24,easy,2018-01-09T21:34:23,16873.0,2018-01-15T23:42:55
25,28182,2018-01-11T10:12:20,easy,2018-01-11T18:44:45,16865.0,2018-01-12T02:46:01
27,28207,2018-01-11T16:27:37,easy,2018-01-11T21:10:51,16867.0,2018-01-12T21:00:24
31,28247,2018-01-12T10:01:12,easy,2018-01-12T17:23:50,16884.0,2018-01-18T18:32:05
...,...,...,...,...,...,...
1581,47498,2018-12-25T19:55:34,easy,2018-12-26T14:20:53,18441.0,2019-01-02T03:48:19
1583,47525,2018-12-26T12:25:19,easy,2018-12-26T17:17:46,18428.0,2018-12-29T23:21:36
1594,47665,2018-12-28T18:51:35,easy,2018-12-28T22:04:40,18438.0,2018-12-31T17:13:00
1595,47671,2018-12-28T20:48:59,easy,2018-12-29T04:26:01,18429.0,2018-12-30T01:39:50


## 5.1.2
##### Рассмотрим данные по уровню сложности - medium

In [56]:
medium_level_to_purchase = merged2018_registration_level_purchase[merged2018_registration_level_purchase['selected_level'] == 'medium']

medium_level_to_purchase

,user_id,registration_time,selected_level,level_choice_time,purchase_id,purchase_time
8,27973,2018-01-06T22:01:44,medium,2018-01-07T05:29:30,16869.0,2018-01-13T21:50:00
10,27981,2018-01-07T08:09:09,medium,2018-01-07T10:46:14,16852.0,2018-01-07T23:20:25
13,28010,2018-01-07T22:19:23,medium,2018-01-08T00:00:52,16858.0,2018-01-10T05:32:47
15,28020,2018-01-08T10:30:32,medium,2018-01-08T14:47:35,16863.0,2018-01-11T21:43:03
16,28026,2018-01-08T13:46:31,medium,2018-01-08T19:29:13,16857.0,2018-01-09T20:51:27
...,...,...,...,...,...,...
1592,47647,2018-12-28T12:14:59,medium,2018-12-28T14:56:29,18442.0,2019-01-02T23:26:26
1593,47659,2018-12-28T17:04:29,medium,2018-12-29T05:10:29,18439.0,2018-12-31T17:13:54
1597,47712,2018-12-29T18:21:28,medium,2018-12-29T22:44:06,18434.0,2018-12-31T09:05:20
1598,47732,2018-12-30T14:20:21,medium,2018-12-31T09:59:00,18452.0,2019-01-06T07:14:25


## 5.1.3
##### Рассмотрим данные по уровню сложности - hard

In [57]:
hard_level_to_purchase = merged2018_registration_level_purchase[merged2018_registration_level_purchase['selected_level'] == 'hard']

hard_level_to_purchase

,user_id,registration_time,selected_level,level_choice_time,purchase_id,purchase_time
0,27845,2018-01-02T01:35:56,hard,2018-01-02T06:19:18,16845.0,2018-01-03T18:53:43
1,27865,2018-01-03T11:14:57,hard,2018-01-04T05:56:32,16846.0,2018-01-04T14:46:10
3,27910,2018-01-05T10:45:33,hard,2018-01-05T11:59:50,16849.0,2018-01-07T12:11:34
4,27911,2018-01-05T10:48:24,hard,2018-01-05T17:39:02,16848.0,2018-01-07T08:19:12
5,27940,2018-01-05T23:41:24,hard,2018-01-06T00:32:47,16850.0,2018-01-07T13:16:41
...,...,...,...,...,...,...
1574,47425,2018-12-24T10:14:07,hard,2018-12-24T17:57:30,18420.0,2018-12-27T21:18:31
1576,47459,2018-12-25T06:18:38,hard,2018-12-25T13:27:39,18415.0,2018-12-26T19:55:41
1579,47477,2018-12-25T13:06:23,hard,2018-12-25T17:24:23,18413.0,2018-12-26T14:22:40
1582,47520,2018-12-26T11:20:31,hard,2018-12-27T03:49:48,18435.0,2018-12-31T14:53:41


##### Проверим правильность разделения пользователей на группы, просуммировав длины строк всех выделенных групп по выбранному уровню сложности (easy / medium / hard) :

In [58]:
len(easy_level_to_purchase) + len(medium_level_to_purchase) + len(hard_level_to_purchase) == len(merged2018_registration_level_purchase)

True

# 5.2

##### Сделаем столбец 
#### timedelta, 
в котором посчитаем разницу между временем выбора уровня сложности
### level_choice_time
и временем покупки платных пакетов вопросов
### purchase_time

In [59]:
merged2018_registration_level_purchase['timedelta'] = pd.to_datetime(merged2018_registration_level_purchase['purchase_time'], format = '%Y-%m-%dT%H:%M:%S') - pd.to_datetime(merged2018_registration_level_purchase['level_choice_time'], format = '%Y-%m-%dT%H:%M:%S')

merged2018_registration_level_purchase

,user_id,registration_time,selected_level,level_choice_time,purchase_id,purchase_time,timedelta
0,27845,2018-01-02T01:35:56,hard,2018-01-02T06:19:18,16845.0,2018-01-03T18:53:43,1 days 12:34:25
1,27865,2018-01-03T11:14:57,hard,2018-01-04T05:56:32,16846.0,2018-01-04T14:46:10,0 days 08:49:38
2,27884,2018-01-04T11:50:43,easy,2018-01-04T16:18:39,16854.0,2018-01-08T19:37:34,4 days 03:18:55
3,27910,2018-01-05T10:45:33,hard,2018-01-05T11:59:50,16849.0,2018-01-07T12:11:34,2 days 00:11:44
4,27911,2018-01-05T10:48:24,hard,2018-01-05T17:39:02,16848.0,2018-01-07T08:19:12,1 days 14:40:10
...,...,...,...,...,...,...,...
1595,47671,2018-12-28T20:48:59,easy,2018-12-29T04:26:01,18429.0,2018-12-30T01:39:50,0 days 21:13:49
1596,47687,2018-12-29T09:49:24,easy,2018-12-29T11:37:35,18432.0,2018-12-30T09:05:28,0 days 21:27:53
1597,47712,2018-12-29T18:21:28,medium,2018-12-29T22:44:06,18434.0,2018-12-31T09:05:20,1 days 10:21:14
1598,47732,2018-12-30T14:20:21,medium,2018-12-31T09:59:00,18452.0,2019-01-06T07:14:25,5 days 21:15:25


## 5.2.1 - 5.2.3

### Теперь сделаем столбцы timedelta для каждого выбранного уровня сложности (easy / medium / hard) и посчитаем разницу между временем выбора каждого уровня сложности и временем покупки платных пакетов вопросов:

## 5.2.1

##### Сделаем столбец timedelta, 
в котором посчитаем разницу между временем выбора легкого уровня сложности 
##### (easy)
##### level_choice_time
и временем покупки платных пакетов вопросов 
##### purchase_time

In [60]:
easy_level_to_purchase['timedelta'] = pd.to_datetime(easy_level_to_purchase['purchase_time'], format = '%Y-%m-%dT%H:%M:%S') - pd.to_datetime(easy_level_to_purchase['level_choice_time'], format = '%Y-%m-%dT%H:%M:%S')

easy_level_to_purchase

,user_id,registration_time,selected_level,level_choice_time,purchase_id,purchase_time,timedelta
2,27884,2018-01-04T11:50:43,easy,2018-01-04T16:18:39,16854.0,2018-01-08T19:37:34,4 days 03:18:55
20,28090,2018-01-09T19:31:24,easy,2018-01-09T21:34:23,16873.0,2018-01-15T23:42:55,6 days 02:08:32
25,28182,2018-01-11T10:12:20,easy,2018-01-11T18:44:45,16865.0,2018-01-12T02:46:01,0 days 08:01:16
27,28207,2018-01-11T16:27:37,easy,2018-01-11T21:10:51,16867.0,2018-01-12T21:00:24,0 days 23:49:33
31,28247,2018-01-12T10:01:12,easy,2018-01-12T17:23:50,16884.0,2018-01-18T18:32:05,6 days 01:08:15
...,...,...,...,...,...,...,...
1581,47498,2018-12-25T19:55:34,easy,2018-12-26T14:20:53,18441.0,2019-01-02T03:48:19,6 days 13:27:26
1583,47525,2018-12-26T12:25:19,easy,2018-12-26T17:17:46,18428.0,2018-12-29T23:21:36,3 days 06:03:50
1594,47665,2018-12-28T18:51:35,easy,2018-12-28T22:04:40,18438.0,2018-12-31T17:13:00,2 days 19:08:20
1595,47671,2018-12-28T20:48:59,easy,2018-12-29T04:26:01,18429.0,2018-12-30T01:39:50,0 days 21:13:49


## 5.2.2

##### Сделаем столбец timedelta, 
в котором посчитаем разницу между временем выбора среднего уровня сложности 
##### (medium)
##### level_choice_time
и временем покупки платных пакетов вопросов
##### purchase_time

In [61]:
medium_level_to_purchase['timedelta'] = pd.to_datetime(medium_level_to_purchase['purchase_time'], format = '%Y-%m-%dT%H:%M:%S') - pd.to_datetime(medium_level_to_purchase['level_choice_time'], format = '%Y-%m-%dT%H:%M:%S')

medium_level_to_purchase

,user_id,registration_time,selected_level,level_choice_time,purchase_id,purchase_time,timedelta
8,27973,2018-01-06T22:01:44,medium,2018-01-07T05:29:30,16869.0,2018-01-13T21:50:00,6 days 16:20:30
10,27981,2018-01-07T08:09:09,medium,2018-01-07T10:46:14,16852.0,2018-01-07T23:20:25,0 days 12:34:11
13,28010,2018-01-07T22:19:23,medium,2018-01-08T00:00:52,16858.0,2018-01-10T05:32:47,2 days 05:31:55
15,28020,2018-01-08T10:30:32,medium,2018-01-08T14:47:35,16863.0,2018-01-11T21:43:03,3 days 06:55:28
16,28026,2018-01-08T13:46:31,medium,2018-01-08T19:29:13,16857.0,2018-01-09T20:51:27,1 days 01:22:14
...,...,...,...,...,...,...,...
1592,47647,2018-12-28T12:14:59,medium,2018-12-28T14:56:29,18442.0,2019-01-02T23:26:26,5 days 08:29:57
1593,47659,2018-12-28T17:04:29,medium,2018-12-29T05:10:29,18439.0,2018-12-31T17:13:54,2 days 12:03:25
1597,47712,2018-12-29T18:21:28,medium,2018-12-29T22:44:06,18434.0,2018-12-31T09:05:20,1 days 10:21:14
1598,47732,2018-12-30T14:20:21,medium,2018-12-31T09:59:00,18452.0,2019-01-06T07:14:25,5 days 21:15:25


## 5.2.3

##### Сделаем столбец timedelta, 
в котором посчитаем разницу между временем выбора сложного уровня сложности 
##### (hard)
##### level_hard_time
и временем покупки платных пакетов вопросов
##### purchase_time

In [62]:
hard_level_to_purchase['timedelta'] = pd.to_datetime(hard_level_to_purchase['purchase_time'], format = '%Y-%m-%dT%H:%M:%S') - pd.to_datetime(hard_level_to_purchase['level_choice_time'], format = '%Y-%m-%dT%H:%M:%S')

hard_level_to_purchase

,user_id,registration_time,selected_level,level_choice_time,purchase_id,purchase_time,timedelta
0,27845,2018-01-02T01:35:56,hard,2018-01-02T06:19:18,16845.0,2018-01-03T18:53:43,1 days 12:34:25
1,27865,2018-01-03T11:14:57,hard,2018-01-04T05:56:32,16846.0,2018-01-04T14:46:10,0 days 08:49:38
3,27910,2018-01-05T10:45:33,hard,2018-01-05T11:59:50,16849.0,2018-01-07T12:11:34,2 days 00:11:44
4,27911,2018-01-05T10:48:24,hard,2018-01-05T17:39:02,16848.0,2018-01-07T08:19:12,1 days 14:40:10
5,27940,2018-01-05T23:41:24,hard,2018-01-06T00:32:47,16850.0,2018-01-07T13:16:41,1 days 12:43:54
...,...,...,...,...,...,...,...
1574,47425,2018-12-24T10:14:07,hard,2018-12-24T17:57:30,18420.0,2018-12-27T21:18:31,3 days 03:21:01
1576,47459,2018-12-25T06:18:38,hard,2018-12-25T13:27:39,18415.0,2018-12-26T19:55:41,1 days 06:28:02
1579,47477,2018-12-25T13:06:23,hard,2018-12-25T17:24:23,18413.0,2018-12-26T14:22:40,0 days 20:58:17
1582,47520,2018-12-26T11:20:31,hard,2018-12-27T03:49:48,18435.0,2018-12-31T14:53:41,4 days 11:03:53


# 5.3

##### Усреднив с помощью функции mean() 
### значения в столбце timedelta, 
мы получим 
##### среднее время
, которое проходит 
##### между выбором уровня сложности (level_choice) и покупкой платных пакетов вопросов (purchase) :

In [63]:
merged2018_registration_level_purchase['timedelta'].mean()

# Между выбором уровня сложности и покупкой платных пакетов вопросов в среднем проходит 3 дня 17 часов 52 минуты

Timedelta('3 days 17:52:17.719375')

## 5.3.1
##### среднее время, которое проходит между выбором легкого уровня сложности (level_choice: easy) и покупкой платных пакетов вопросов (purchase) :

In [64]:
easy_level_to_purchase['timedelta'].mean()

# Между выбором лёгкого уровня сложности и покупкой платных пакетов вопросов в среднем проходит 3 дня 14 часов 58 минут

Timedelta('3 days 14:58:52.941798941')

## 5.3.2
##### среднее время, которое проходит между выбором среднего уровня сложности (level_choice: medium) и покупкой платных пакетов вопросов (purchase) :

In [65]:
medium_level_to_purchase['timedelta'].mean()

# Между выбором среднего уровня сложности и покупкой платных пакетов вопросов в среднем проходит 3 дня 23 часов 14 минут

Timedelta('3 days 23:14:13.165118679')

## 5.3.3
##### среднее время, которое проходит между выбором сложного уровня сложности (level_choice: hard) и покупкой платных пакетов вопросов (purchase) :

In [66]:
hard_level_to_purchase['timedelta'].mean()

# Между выбором среднего уровня сложности и покупкой платных пакетов вопросов в среднем проходит 3 дня 7 часов 20 минут

Timedelta('3 days 07:20:41.420814479')

# 5.4 - Вопрос самостоятельной работы 2. 

Мы видим, что покупка платных пакетов вопросов происходит :

##### быстрее при выборе самого сложного уровня сложности (level_choice: hard) - в среднем 3 дня 7 часов 20 минут;

##### немного больше времени наблюдается при выборе легкого уровня сложности (level_choice: easy) - в среднем 3 дня 14 часов 58 минут; 

##### больше всего времени до покупки проходит при выборе среднего уровня сложности (level_choice: medium) - в среднем 3 дня 23 часов 14 минут.


Таким образом, определённо существует разница между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности: пользователи с уровнем сложности hard склонны к оплате пакета платных вопросов быстрее (в среднем 3 дня 7 часов 20 минут), нежели чем пользователи других групп, среди которых пользователи с уровнем сложности easy склонны к оплате (в среднем 3 дня 14 часов 58 минут) быстрее чем пользователи с уровнем сложности medium (в среднем 3 дня 23 часов 14 минут).

Можно предположить, что пользователи, предпочитающие сложный уровень склонны к оплате быстрее, чем пользователи, выбравшие легкий и средний уровни сложности из-за большей вовлечённости в процесс игры. 

Проверим нашу гипотезу с помощью анализа дополнительных параметров >>>

# 5.4

Воспользуемся методом 
### describe(), 
чтобы понять некоторые важные параметры для объединённого по уровню сложности датафрейма, а также раздельно для датафреймов по каждому уровню сложности :

In [67]:
merged2018_registration_level_purchase['timedelta'].describe()

count                         1600
mean        3 days 17:52:17.719375
std      2 days 04:37:14.656359486
min                0 days 00:49:20
25%         1 days 21:27:33.750000
50%                3 days 12:59:27
75%         5 days 09:47:33.750000
max               10 days 18:35:09
Name: timedelta, dtype: object

## 5.4.1

In [68]:
easy_level_to_purchase['timedelta'].describe()

count                          189
mean     3 days 14:58:52.941798941
std      2 days 07:06:35.644097504
min                0 days 00:49:20
25%                1 days 17:18:56
50%                3 days 06:03:50
75%                5 days 06:58:18
max               10 days 18:35:09
Name: timedelta, dtype: object

## 5.4.2

In [69]:
medium_level_to_purchase['timedelta'].describe()

count                          969
mean     3 days 23:14:13.165118679
std      2 days 06:18:57.618467109
min                0 days 04:18:12
25%                2 days 01:20:07
50%                3 days 19:53:19
75%                5 days 16:07:19
max               10 days 13:51:01
Name: timedelta, dtype: object

## 5.4.3

In [70]:
hard_level_to_purchase['timedelta'].describe()

count                          442
mean     3 days 07:20:41.420814479
std      1 days 21:43:52.953292605
min                0 days 03:26:45
25%         1 days 14:57:23.500000
50%         3 days 03:13:57.500000
75%         4 days 19:16:00.250000
max                8 days 01:18:13
Name: timedelta, dtype: object

# 5.5

### К ЧЕМУ МЫ ПРИШЛИ?


##### для всех уровней сложности - (1600 пользователей) :
1-й квартиль составляет 1 день 21 час 27 минут, т.е. четверть пользователей тратит меньше 1 дня 21 часа 27 минут между выбором уровня сложности и покупкой платных пакетов вопросов;
- медиана (второй квартиль) — 3 дня 12 часов 59 минут, т.е. половина всех пользователей тратит между выбором уровня сложности и покупкой платных пакетов вопросов менее 3 дней 12 часов 59 минут.



##### для легкого уровня сложности (level_choice: easy) : 
1-й квартиль составляет 1 день 17 час 18 минут, т.е. четверть пользователей тратит меньше 1 дня 17 часа 18 минут между выбором уровня сложности и покупкой платных пакетов вопросов;
- медиана (второй квартиль) — 3 дня 6 часов 3 минут, т.е. половина всех пользователей тратит между выбором уровня сложности и покупкой платных пакетов вопросов менее 3 дней 6 часов 3 минут.

##### для среднего уровня сложности (level_choice: medium) : 
1-й квартиль составляет 2 дня 1 час 20 минут, т.е. четверть пользователей тратит меньше 2 дня 1 час 20 минут между выбором уровня сложности и покупкой платных пакетов вопросов;
- медиана (второй квартиль) — 3 дня 19 часов 53 минут, т.е. половина всех пользователей тратит между выбором уровня сложности и покупкой платных пакетов вопросов менее 3 дней 19 часов 53 минут.

##### для сложного уровня сложности (level_choice: hard) : 
1-й квартиль составляет 1 день 14 часов 57 минут, т.е. четверть пользователей тратит меньше 1 дня 14 часов 57 минут между выбором уровня сложности и покупкой платных пакетов вопросов;
- медиана (второй квартиль) — 3 дня 3 часа 13 минут, т.е. половина всех пользователей тратит между выбором уровня сложности и покупкой платных пакетов вопросов менее 3 дней 3 часов 13 минут.



Анализ дополнительных параметров подтверждает нашу гипотезу о том, что время оплаты зависит от выбранного пользователем уровня сложности: пользователи, предпочитающие сложный уровень склонны к оплате быстрее, нежели чем пользователи, выбравшие легкий и средний уровни сложности - это видно по данным двух крайних квартилей и медианы, а также значительно меньшим, чем у других групп, показателям максимального значения (8 дней 1 час 18 минут) и стандартного отклонения (1 день 21 час 43 минуты).

Однако, стоит заметить, что количество пользователей, выбравших сложный уровень сложности (442) значительно меньше, чем число пользователей, выбравших средний уровень сложности (969), но больше чем число пользователей, выбравших легкий уровень сложности (189).

##### Таким образом, мы можем сосредоточится не на оптимизации среднего времени, т.к. среднее будет увеличено за счёт пользователей, которые долго проходят этап, а на том, чтобы оптимизировать медианное время, то есть время, которое тратит половина пользователей, а особенно медианное время пользователей с сложным и средним уровнем сложности.

##### Также, можем отдельно выделить тех пользователей, у которых время выше медианного, чтобы исследовать их более детально и понять, что мешает им быстрее дойти до этапа покупки платных пакетов вопросов.

## Проверка аналитической гипотезы

### Вопрос самостоятельной работы 1. 
##### Настало время понять, зависит ли вероятность оплаты от выбранного пользователем уровня сложности. 


Для этого определим, существует ли различие в частоте и средней величине оплат между тремя группами пользователей:

    - пользователи, которые выбрали уровень сложности - easy;

    - пользователи, которые выбрали уровень сложности - medium;

    - пользователи, которые выбрали уровень сложности - hard.
    

## 6.1.1 - 6.1.3
##### Сначала определим группы зарегистрированных пользователей, которые выбрали соответствующий уровень сложности (level_choice): easy / medium / hard  

## 6.2.1 - 6.2.3
##### Затем, по каждой группе пользователей определим количество пользователей, которые оплатили пакет платных вопросов и рассчитаем процент пользователей, которые оплатили пакет платных вопросов в каждой группе

## 6.3.1 - .6.3.3
##### Дальше найдём средний размер платежа для этих пользователей в каждой группе:


# 6.1

In [112]:
# Всего, зарегистрированных в 2018 г. пользователей, выбравших все уровни сложности и оплативших пакет платных вопросов :

merged2018_registration_level['user_id'].nunique()

8342

## 6.1.1

In [71]:
# 29.35% от общего количества пользователей, выбравших какой-либо уровень сложности:

registration_to_easy_level['user_id'].nunique()

2448

In [72]:
easy_level_to_purchase['user_id'].nunique()

189

## 6.1.2

In [73]:
# 55.68% от общего количества пользователей, выбравших какой-либо уровень сложности:

registration_to_medium_level['user_id'].nunique()

4645

In [74]:
medium_level_to_purchase['user_id'].nunique()

969

## 6.1.3

In [75]:
# 14.97% от общего количества пользователей, выбравших какой-либо уровень сложности:

registration_to_hard_level['user_id'].nunique()

1249

In [76]:
hard_level_to_purchase['user_id'].nunique()

442

# 6.2

Посмотрим процент пользователей по всем уровням сложности (easy, medium, hard), которые оплатили пакет платных вопросов :

In [113]:
# превращаем датафрейм в список:
all_level_list = merged2018_registration_level.user_id.to_list()

# формируем датафрейм all_level_purchases, в котором будут данные по оплатам пользователей со всеми уровнями сложности:
all_level_purchases = purchases_p[purchases_p['user_id'].isin(all_level_list)]

# рассчитываем процент пользователей со всеми уровнями сложности, которые оплатили пакет платных вопросов:
percent_of_all_level_purchases = all_level_purchases['user_id'].nunique() / len(all_level_list)

print('Процент пользователей со всеми уровнями сложности, которые оплатили пакет платных вопросов: {:.2%}'.format(percent_of_all_level_purchases))

Процент пользователей со всеми уровнями сложности, которые оплатили пакет платных вопросов: 19.18%


Теперь сделаем отдельные датафреймы:  
##### easy_purchase
##### medium_purchase
##### easy_purchase
, в которых будут данные по оплатам пользователей, выбравших соответствующий уровень сложности (easy / medium / hard) и определим процент оплативших пользователей в каждой группе :


## 6.2.1

In [77]:
# превращаем датафрейм в список:
easy_list = registration_to_easy_level.user_id.to_list()

# формируем датафрейм easy_purchases, в котором будут данные по оплатам пользователей с уровнем сложности easy
easy_purchases = purchases_p[purchases_p['user_id'].isin(easy_list)]

# рассчитываем процент пользователей с уровнем сложности easy, которые оплатили пакет платных вопросов
percent_of_easy_purchases = easy_purchases['user_id'].nunique() / len(easy_list)

print('Процент пользователей с уровнем сложности easy, которые оплатили пакет платных вопросов: {:.2%}'.format(percent_of_easy_purchases))

Процент пользователей с уровнем сложности easy, которые оплатили пакет платных вопросов: 7.72%


## 6.2.2

In [78]:
# превращаем датафрейм в список:
medium_list = registration_to_medium_level.user_id.to_list()

# формируем датафрейм medium_purchases, в котором будут данные по оплатам пользователей c уровнем сложности medium
medium_purchases = purchases_p[purchases_p['user_id'].isin(medium_list)]

# рассчитываем процент пользователей с уровнем сложности medium, которые оплатили пакет платных вопросов
percent_of_medium_purchases = medium_purchases['user_id'].nunique() / len(medium_list)

print('Процент пользователей с уровнем сложности medium, которые оплатили пакет платных вопросов: {:.2%}'.format(percent_of_medium_purchases))

Процент пользователей с уровнем сложности medium, которые оплатили пакет платных вопросов: 20.86%


## 6.2.3

In [79]:
# превращаем датафрейм в список:
hard_list = registration_to_hard_level.user_id.to_list()

# формируем датафрейм hard_purchases, в котором будут данные по оплатам пользователей c уровнем сложности hard
hard_purchases = purchases_p[purchases_p['user_id'].isin(hard_list)]

# рассчитываем процент пользователей с уровнем сложности hard, которые оплатили пакет платных вопросов
percent_of_hard_purchases = hard_purchases['user_id'].nunique() / len(hard_list)

print('Процент пользователей с уровнем сложности hard, которые оплатили пакет платных вопросов: {:.2%}'.format(percent_of_hard_purchases))

Процент пользователей с уровнем сложности hard, которые оплатили пакет платных вопросов: 35.39%


## 6.3.1

In [80]:
# найдём средний размер платежа для пользователей с уровнем сложности easy:

easy_purchases['amount'].mean()

114.94708994708995

In [97]:
easy_purchases['amount'].describe()

count    189.00000
mean     114.94709
std       51.74608
min       25.00000
25%      100.00000
50%      100.00000
75%      150.00000
max      250.00000
Name: amount, dtype: float64

## 6.3.2

In [81]:
# найдём средний размер платежа для пользователей с уровнем сложности medium:

medium_purchases['amount'].mean()

109.52012383900929

In [110]:
medium_purchases['amount'].describe()

count    969.000000
mean     109.520124
std       55.043201
min       25.000000
25%       50.000000
50%      100.000000
75%      150.000000
max      300.000000
Name: amount, dtype: float64

## 6.3.3

In [91]:
# найдём средний размер платежа для пользователей с уровнем сложности hard:

hard_purchases['amount'].mean()

111.59502262443439

In [99]:
hard_purchases['amount'].describe()

count    442.000000
mean     111.595023
std       55.173413
min       25.000000
25%       50.000000
50%      100.000000
75%      150.000000
max      300.000000
Name: amount, dtype: float64

# 6.4

##### Для сравнения, определим всех зарегистрированных в 2018 году пользователей, не выбиравших уровень сложности, но оплативших пакет платных услуг и рассчитываем процент оплативших пользователей в этой группе :

In [111]:
# превращаем датафрейм в список:
registered_list = registration2018.user_id.to_list()

# формируем датафрейм registered_purchases, в котором будут данные по оплатам зарегистрированных пользователей не выбиравших уровень сложности
registered_purchases = purchases_p[purchases_p['user_id'].isin(registered_list)]

# рассчитываем процент зарегистрированных пользователей не выбиравших уровень сложности, которые оплатили пакет платных вопросов
percent_of_registered_purchases = registered_purchases['user_id'].nunique() / len(registered_list)

print('Процент зарегистрированных пользователей не выбиравших уровень сложности, которые оплатили пакет платных вопросов: {:.2%}'.format(percent_of_registered_purchases))

Процент зарегистрированных пользователей не выбиравших уровень сложности, которые оплатили пакет платных вопросов: 8.03%


# 6.5 - Вопрос самостоятельной работы 1. 

##### Полученные результаты подтверждают нашу гипотезу о том, что вероятность оплаты зависит от выбранного пользователем уровня сложности: пользователи, предпочитающие уровень hard больше склонны к оплате (35.39%), нежели чем пользователи, выбравшие medium (20.86%) и easy (7.72%) уровни сложности. 

Однако, стоит заметить, что средний размер платежа для пользователей с уровнем easy (114 рублей 94 копейки) ненамного, но больше среднего размера платежа для пользователей с уровнем hard (111 рублей 59 копейки) и среднего размера платежа для пользователей с уровнем medium (109 рублей 52 копейки) - это происходит из-за того, что у 25% пользователей (1-ый квартиль) с уровнем сложности easy размер платежа (100 рублей), больше, чем у 25% пользователей (1-ый квартиль) с уровнем сложности medium и hard (50 рублей, соответственно); при этом, значения 2-го и 3-го квартиля у всех групп пользователей совпадает (100 рублей и 150 рублей, соответственно). 

Таким образом, несмотря на небольшое превосходство среднего размера платежа пользователей с уровнем сложности easy, можно сделать следующий вывод: чем сложнее уровень сложности, тем выше вероятность оплаты/покупки пользователями пакета платных вопросов. Если же разница между средними платежами является материальной, то  между выбором уровня сложности и покупкой пакета платных вопросов есть определённая, но не сильная зависимость.

Также, стоит обратить внимание на то, что процент пользователей со всеми уровнями сложности (easy, medium, hard), которые оплатили пакет платных вопросов (19.18%) более чем в 2 раза превышает тех пользователей, которые также оплатили пакет платных вопросов, но не выбирали какой-либо уровень сложности (8.03%). 